In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Osteoporosis"
cohort = "GSE80614"

# Input paths
in_trait_dir = "../../input/GEO/Osteoporosis"
in_cohort_dir = "../../input/GEO/Osteoporosis/GSE80614"

# Output paths
out_data_file = "../../output/preprocess/Osteoporosis/GSE80614.csv"
out_gene_data_file = "../../output/preprocess/Osteoporosis/gene_data/GSE80614.csv"
out_clinical_data_file = "../../output/preprocess/Osteoporosis/clinical_data/GSE80614.csv"
json_path = "../../output/preprocess/Osteoporosis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression analyses of the differentiating hMSC into Osteoblasts and Adipocytes."
!Series_summary	"Age-related skeletal degeneration in patients with osteoporosis is characterized by decreased bone mass and occurs concomitant with an increase in bone marrow adipocytes. Using microarray expression profiling with high temporal resolution, we identified gene regulatory events in early stages of osteogenic and adipogenic lineage commitment of human mesenchymal stromal cells (hMSCs). Data analysis reveal three distinct phases when cells adopt a committed expression phenotype: initiation of differentiation (0-3h, Phase I), lineage-acquisition (6-24h, Phase II) and early lineage-progression (48-96h, Phase III). Upstream regulator analysis identifies 34 transcription factors (TFs) in Phase I with a role in hMSC differentiation. Interestingly, expression levels of identified TFs did not always change and indicate additional post-transcriptional regula

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import os
import json
import pandas as pd
from typing import Optional, Callable, Dict, Any

# 1. Gene Expression Data Availability
# From the background information, we can see this dataset is about gene expression analyses
# of MSCs differentiating into osteoblasts and adipocytes, which indicates gene expression data.
is_gene_available = True

# 2. Variable Availability and Data Type Conversion

# 2.1 Data Availability
# Looking at the sample characteristics dictionary:
# For trait: There's no direct mention of osteoporosis status in the sample data
# For age: Found in key 1 - "age: 19 years", "age: 19" 
# For gender: Found in key 0 - "gender: Male"

# Examining if these are constant features:
# For trait: Not directly available in sample characteristics
# For age: Value appears to be constant at 19 years
# For gender: Value appears to be constant as Male

trait_row = None  # Osteoporosis status not directly available in sample characteristics
age_row = None    # Age is constant (19 years), not useful for association study
gender_row = None # Gender is constant (Male), not useful for association study

# 2.2 Data Type Conversion
# Since all variables are not available or constant, we won't need conversion functions
# but we still need to define trait conversion for the validate_and_save_cohort_info function

def convert_trait(value):
    """Convert trait value to binary (0 for control, 1 for case)."""
    if value is None:
        return None
    value = value.lower().strip()
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    if value in ["osteoporosis", "case", "yes", "positive", "1", "true"]:
        return 1
    elif value in ["control", "no", "negative", "0", "false", "normal"]:
        return 0
    return None

# 3. Save Metadata
# Initial filtering and saving information
is_trait_available = trait_row is not None

validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is None, we skip this step as instructed


False

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651199', 'ILMN_1651209',
       'ILMN_1651210', 'ILMN_1651221', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651230', 'ILMN_1651232', 'ILMN_1651235', 'ILMN_1651236',
       'ILMN_1651237', 'ILMN_1651238', 'ILMN_1651249', 'ILMN_1651253',
       'ILMN_1651254', 'ILMN_1651259', 'ILMN_1651260', 'ILMN_1651262'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These identifiers start with "ILMN_" which indicates they are Illumina probe IDs, not human gene symbols.
# Illumina IDs need to be mapped to gene symbols for meaningful biological interpretation.
# The format "ILMN_" followed by numbers is characteristic of Illumina BeadArray microarray platforms.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['ILMN_1725881', 'ILMN_1910180', 'ILMN_1804174', 'ILMN_1796063', 'ILMN_1811966'], 'nuID': ['rp13_p1x6D80lNLk3c', 'NEX0oqCV8.er4HVfU4', 'KyqQynMZxJcruyylEU', 'xXl7eXuF7sbPEp.KFI', '9ckqJrioiaej9_ajeQ'], 'Species': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Source': ['RefSeq', 'Unigene', 'RefSeq', 'RefSeq', 'RefSeq'], 'Search_Key': ['ILMN_44919', 'ILMN_127219', 'ILMN_139282', 'ILMN_5006', 'ILMN_38756'], 'Transcript': ['ILMN_44919', 'ILMN_127219', 'ILMN_139282', 'ILMN_5006', 'ILMN_38756'], 'ILMN_Gene': ['LOC23117', 'HS.575038', 'FCGR2B', 'TRIM44', 'LOC653895'], 'Source_Reference_ID': ['XM_933824.1', 'Hs.575038', 'XM_938851.1', 'NM_017583.3', 'XM_936379.1'], 'RefSeq_ID': ['XM_933824.1', nan, 'XM_938851.1', 'NM_017583.3', 'XM_936379.1'], 'Unigene_ID': [nan, 'Hs.575038', nan, nan, nan], 'Entrez_Gene_ID': [23117.0, nan, 2213.0, 54765.0, 653895.0], 'GI': [89040007.0, 10437021.0, 88952550.0, 29029528.0, 89033487.0], 'Access

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine which columns to use for mapping
# From the preview, we can see:
# - 'ID' column contains the probe identifiers (ILMN_XXXXXX) matching our gene expression data
# - 'Symbol' column contains the gene symbols we want to map to

# 2. Get a gene mapping dataframe by extracting these two columns
gene_mapping = get_gene_mapping(gene_annotation, 'ID', 'Symbol')

# 3. Convert probe-level measurements to gene expression data
# This applies the mapping and handles many-to-many relations between probes and genes
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Print the shape of the mapped gene data to verify transformation
print(f"Shape of gene expression data after mapping: {gene_data.shape}")

# Display the first few gene symbols to confirm mapping worked
print("First few gene symbols after mapping:")
print(gene_data.index[:10])


Shape of gene expression data after mapping: (19120, 69)
First few gene symbols after mapping:
Index(['A1BG', 'A1CF', 'A26A1', 'A26B1', 'A26C1B', 'A26C3', 'A2BP1', 'A2M',
       'A2ML1', 'A3GALT2'],
      dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# Since we determined in Step 2 that trait data is not available, we should use non-final validation
# Update the cohort information with is_final=False
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=False
)

print("Data does not contain Osteoporosis trait information and cannot be used for association studies")

Normalized gene data saved to ../../output/preprocess/Osteoporosis/gene_data/GSE80614.csv
Data does not contain Osteoporosis trait information and cannot be used for association studies
